In [20]:
import json
import numpy as np
import requests

url = "http://127.0.0.1:8000/"
# /predict POST -> all 30 attributes
# /conversion POST -> user_id, value
# /resultsAB GET -> get all conversions in JSON
# /resetAB POST -> reset memory of conversions

headers = {
    'Content-Type': 'application/json'
}

In [21]:
N = 1
print("Processing ",N," events...")

Processing  1  events...


In [22]:
for i in range(N):
    # generator of attributes
    attributes = dict()
    attributes['user_id'] = i
    attributes['price'] = round(np.random.normal(2000, 1000),0)
    attributes['product_views_count'] = int(abs(round(np.random.normal(0, 15), 0)))
    attributes['category_view_count'] = int(abs(round(np.random.normal(0, 11), 0)))
    attributes['bought_with_lower_count'] = int(abs(round(np.random.normal(0, 6), 0)))
    attributes['bought_category_count'] = int(abs(round(np.random.normal(0, 7), 0)))
    attributes['time'] = abs(round(np.random.normal(0, 1000), 2))
    city_index = int(round(np.random.uniform(0, 7), 0))
    for i in range(8):
        attributes['one_hot_city' + str(i)] = 1 if i == city_index else 0

    category_index = round(np.random.uniform(0, 14))
    for i in range(15):
        attributes['one_hot_category' + str(i)] = 1 if i == category_index else 0

    # send request for predictions of discount
    json_data = json.dumps(attributes, ensure_ascii=True)
    response = requests.request("POST", url+'predict/', headers=headers, data=json_data)

    #%%

    discount = response.json()['discount']

    #%%

    # generate decision about buying
    import random
    decision = random.random() < (discount/100.0 + 0.1)
    # if buying then post conversion to server
    conversion_dictionary = dict()
    conversion_dictionary['user_id'] = i
    conversion_dictionary['value'] = int(attributes['price']*(0.3-discount)) # TODO change to float
    conversion_data = json.dumps(conversion_dictionary)
    conversion_response = requests.request("POST", url+'conversion/', headers=headers, data=conversion_data)
    print(conversion_response.content)

b'{"id":1,"model_id":0,"value":-21078}'


In [23]:
# after all actions

# get all conversions in JSON
all_conversions = requests.request("GET", url+'resultsAB/')
conversions = all_conversions.json()

# count conversions and print results
value0 = 0
value1 = 0
for conversion in conversions:
    if conversion['model_id'] == 0:
        value0 += conversion['value']
    else:
        value1 += conversion['value']

print("Conversion value for dummy base model: ", value0)
print("Conversion value for tuned model:      ", value0)

Conversion value for dummy base model:  -21078
Conversion value for tuned model:       -21078
